# Directive de "Pré-processing"

In [2]:
Version_Lyam_Neal = 'Roi_Version_Lyam_Neal_Échantillon.mp3'
Data_Lyam_Neal, samplerate_Lyam_Neal = librosa.load(Version_Lyam_Neal, sr=None)
length_Lyam_Neal = Data_Lyam_Neal.shape[0] / samplerate_Lyam_Neal
time_Lyam_Neal = np.linspace(0., length_Lyam_Neal, Data_Lyam_Neal.shape[0])
Data_Lyam_Neal.shape[0]

1890212

In [3]:
Version_Bilal_Hassani = 'Roi_Version_Bilal_Hassani_Échantillon.mp3'
Data_Bilal_Hassani, samplerate_Bilal_Hassani = librosa.load(Version_Bilal_Hassani, sr=None)
length_Bilal_Hassani = Data_Bilal_Hassani.shape[0] / samplerate_Bilal_Hassani
time_Bilal_Hassani = np.linspace(0., length_Bilal_Hassani, Data_Bilal_Hassani.shape[0])
Data_Bilal_Hassani.shape[0]

1890212

### Software to get data into a file

In [4]:
def recup(nomfichier, colonnes, sep = ";"):
    """
    nomfichier : fichier texte .txt ou .csv
    colonnes : type int, nbre de colonnes à recuperer
    sep : type str, separateur de colonnes, ";" par defaut
    RETOUR : type array, tableau de donnees.
    """
    frequences = []#initialisaition de la liste de donnees
    decibel = []
    with open(nomfichier, "r") as contenu:
        for ligne in contenu:
            try : #pour eviter les entêtes
                ligne = ligne.split(sep) #separe les valeurs
                frequences.append(float(ligne[0]))
                decibel.append(float(ligne[1]))#remplissage et conversion en flottant
            except: ValueError
    return (frequences,np.array(decibel)) # conversion en tableau

# Algorithme de Cooley-Tukey

## Algorithme de Moyenne-Glissante

In [5]:
def moyenne_glissante_prof(Ltemps,Lsignal,nmoy):
    Lres=[]
    Ltempsres=[]
    for i in range (0,len(Lsignal)-nmoy):
        res=sum(Lsignal[i:i+nmoy])/nmoy
        Lres.append(res)
        Ltempsres.append(sum(Ltemps[i:i+nmoy])/nmoy)
    return Ltempsres,Lres

## Algorithme de Cooley-Tukey

In [6]:
def ditfft2(x, N, s):
    # Cas de base : si la longueur de la séquence est 1, retourne l'élément lui-même
    if N == 1:
        return [x[0]]
    # Calcul de la FFT des sous-séquences en utilisant le saut `s`
    even_part = ditfft2(x, N // 2, s * 2)  # Partie paire avec un saut de `2s`
    odd_part = ditfft2(x[s:], N // 2, s * 2)  # Partie impaire avec un décalage et saut de `2s`
    # Initialiser une liste pour stocker le résultat final
    result = [0] * N
    for k in range(N // 2):
        # Calcul du facteur twiddle
        t = np.exp(-2j * np.pi * k / N) * odd_part[k]

        # Combinaison des parties paires et impaires pour la FFT finale
        result[k] = even_part[k] + t
        result[k + N // 2] = even_part[k] - t
    return result

# Similarité de Spectre

Ici sont analysé deux séquences d'une même musique qui est ROI de Bilal Hassani, on analyse le refrain.

### Utilisation du bitwise pour l'approche d'une puissance de deux

In [7]:
def puissance_deux_inferieure_rapide(n):
    if n < 1:
        raise ValueError("La valeur doit être supérieure ou égale à 1.")
    return 1 << (n.bit_length() - 1)

### FFT par Cooley-Tukey de la version Lyam_Neal

#### Séléction des données

In [8]:
# Plage de donnée
MIN = 0
N = puissance_deux_inferieure_rapide(min(Data_Lyam_Neal.shape[0],Data_Bilal_Hassani.shape[0]))

#Fenêtre gaussienne
#window = get_window(("gaussian",2.5), N)
#window = get_window("hann", N)
window = np.ones(N)


In [9]:
Restricted_Data_Lyam_Neal = (Data_Lyam_Neal[MIN:MIN+N] * window)# Garder uniquement les premiers N échantillons

FFT_Lyam_Neal = ditfft2(Restricted_Data_Lyam_Neal,N,1)
FFT_Lyam_Neal = np.abs(FFT_Lyam_Neal)[:N // 2]
int_max = max(FFT_Lyam_Neal)
FFT_Lyam_Neal = 20*np.log10(FFT_Lyam_Neal/int_max+1e-12)
Frequencies_Lyam_Neal = np.fft.fftfreq(N, d=1/samplerate_Lyam_Neal)# Génère les fréquences

# Prendre le module (amplitude) de la FFT et garder uniquement la moitié positive des fréquences
Frequencies_Lyam_Neal = Frequencies_Lyam_Neal[:N // 2]

### FFT par Cooley-Tukey de la version Bilal_Hassani

In [10]:
#Fenêtre gaussienne
Restricted_Data_Bilal_Hassani = Data_Bilal_Hassani[MIN:(MIN+N)] * window

FFT_Bilal_Hassani = ditfft2(Restricted_Data_Bilal_Hassani,N,1)
FFT_Bilal_Hassani = np.abs(FFT_Bilal_Hassani)[:N // 2]
int_max = max(FFT_Bilal_Hassani)
FFT_Bilal_Hassani = 20*np.log10(FFT_Bilal_Hassani/int_max+1e-12)
Frequencies_Bilal_Hassani = np.fft.fftfreq(N, d=1/samplerate_Bilal_Hassani)# Génère les fréquences

# Prendre le module (amplitude) de la FFT et garder uniquement la moitié positive des fréquences
Frequencies_Bilal_Hassani = Frequencies_Bilal_Hassani[:N // 2]

## Affichage des données

In [11]:
Graph = ChemPlot("Spectre","Fréquences (Hz)","Amplitudes (Hz)","Variétés")
Graph.add_plot([Frequencies_Lyam_Neal,FFT_Lyam_Neal],"","Version Lyam_Neal",color=c1)
Graph.add_plot([Frequencies_Bilal_Hassani,FFT_Bilal_Hassani],"","Version Bilal_Hassani",color=c4)
Graph.show_browser()

# Version Audacity

In [3]:
Graphe = ChemPlot("Spectre","Fréquences (Hz)","Amplitudes (Hz)","Variétés")
Graphe.add_plot("spectre_Roi_Version_BillalH.txt",";","Version Effeil65",color=c1)
Graphe.add_plot("spectre_Roi_Version_LyamN.txt",";","Version Guetta",color=c4)
Graphe.show_browser()